## Import

In [278]:
import os
import yaml
import warnings
from typing import List

from utils import get_openai_api_key

from crewai import Agent, Task, Crew
from pydantic import BaseModel, Field
from crewai.tools import BaseTool


warnings.filterwarnings('ignore')

#### Get api key

In [279]:
openai_api_key = get_openai_api_key()

## Setup configs

In [280]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Pydantic models

In [281]:
class EventSlot(BaseModel):
    date: str = Field(..., description="The date of the event. The format is: DD/MM/YYYY")
    weekday: str = Field(..., description="The date's week day.")
    startTime: str = Field(..., description="Start time of the slot. The format is: HH:MM")
    endTime: str = Field(..., description="End time of the slot. The format is: HH:MM")
    title: str = Field(..., description="Short title to summarize the event's essential information.")

class MultipleSlots(BaseModel):
    slots: List[EventSlot] = Field(..., description="List of event slots")

## Tools

In [282]:
from datetime import date
from typing import Type


from datetime import datetime

class InputDate(BaseModel):
    date: str = Field(..., description="The date of the event. The format is: DD/MM/YYYY")
    weekday: str = Field(..., description="The date's week day.")


class ValidDateTool(BaseTool):
    name: str = "Valid Date Tool"
    description: str = "Tool that checks if the date and week day are valid"
    args_schema: Type[BaseModel] = InputDate

    def _run(self, date: str, weekday: str):
        try:
            datetime.strptime(date, "%d/%m/%Y")
        except ValueError:
            return f"Error: {str(e)}. Please provide a valid date."
        
        try:
            date_obj = datetime.strptime(date, "%d/%m/%Y")
            correct_weekday = date_obj.strftime("%A")  # Full name (e.g., "Monday")
            correct_weekday.lower().startswith(weekday.lower())
            return "Valid date"
        except ValueError as e:
            return f"Error: {str(e)}. The week day does not corresponds to the date"

In [283]:
validDateTool = ValidDateTool()

## Agents

In [284]:
converter = Agent(
    config=agents_config["converter"],
    inject_date=True,
    date_format="%A, %B %d, %Y",
    tools=[validDateTool])

## Tasks

In [285]:
conversion = Task(
    config=tasks_config["conversion"],
    agent=converter,
    output_pydantic=MultipleSlots
)

## Crew

In [286]:
crew = Crew(
    agents=[converter],
    tasks=[conversion],
    verbose=False
)

## Execution

In [287]:
result = crew.kickoff(inputs={"event": "I need to go to the gym at 9am for an hour for the next four mondays"})

In [291]:
result.pydantic.model_dump()

{'slots': [{'date': '02/10/2025',
   'weekday': 'Wednesday',
   'startTime': '09:00',
   'endTime': '10:00',
   'title': 'Gym Session'},
  {'date': '09/10/2025',
   'weekday': 'Wednesday',
   'startTime': '09:00',
   'endTime': '10:00',
   'title': 'Gym Session'},
  {'date': '16/10/2025',
   'weekday': 'Wednesday',
   'startTime': '09:00',
   'endTime': '10:00',
   'title': 'Gym Session'},
  {'date': '23/10/2025',
   'weekday': 'Wednesday',
   'startTime': '09:00',
   'endTime': '10:00',
   'title': 'Gym Session'}]}

## Cost

In [289]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0006


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,4011,3675,0,336,5
